In [1]:
import os

In [2]:
%pwd

'/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema.name
            
        )

        return model_trainer_config

In [15]:
import pandas as pd
import os
from mlProject import logger
import joblib
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.optimizers import Adam
import tensorflow

In [21]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        ann = Sequential()
        ann.add(Dense(64, activation='relu', input_shape=(train_x.shape[1],)))
        ann.add(Dense(32, activation='relu', input_shape=(train_x.shape[1] - 2,)))
        ann.add(Dense(10, activation='softmax'))
        loss = keras.losses.sparse_categorical_crossentropy
        optimizer = Adam(lr=0.01)
        ann.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
        ann.fit(train_x, train_y, epochs=5)

        joblib.dump(ann, os.path.join(self.config.root_dir, self.config.model_name))



In [22]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-19 19:01:30,374: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-19 19:01:30,376: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-19 19:01:30,379: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-19 19:01:30,380: INFO: common: created directory at: artifacts]
[2023-09-19 19:01:30,381: INFO: common: created directory at: artifacts/model_trainer]
[2023-09-19 19:01:30,466: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.]
[2023-09-19 19:01:30,467: WARNING: optimizer: `lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.]
[2023-09-19 19:01:30,468: WARNING: __init__: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `